In [ ]:
!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.8/258.8 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.7 MB/s eta 0:00:00


In [ ]:
import wandb
import numpy as np
from types import SimpleNamespace
import random

In [ ]:
!wandb login

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
# You need to define a config file in the form of dictionary or yaml
sweep_config = {
    'method': 'grid',
    'name' : 'sweep cross entropy',
    'metric': {
      'name': 'val_accuracy',
      'goal': 'maximize'
    },
    'parameters': {
        'epochs': {
            'values': [5,10]
        },
         'hidden_size':{
            'values':[32,64,128]
        },
        'activation': {
            'values': ['sigmoid','tanh','relu']
        },
        'loss': {
            'values': ['cross_entropy','squared_loss']
        },


    }
}

sweep_id = wandb.sweep(sweep=sweep_config, project='DL_TA(Ass_1)')

In [25]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from keras.datasets import fashion_mnist
import math

In [26]:
# Load MNIST data using Keras
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
print(x_train.shape)
x_train,x_val, y_train, y_val=train_test_split(x_train,y_train, test_size=0.2,shuffle=True,random_state=42)
print(x_train.shape)

# Preprocess the data
x_train = x_train.astype('float128') / 255.0
x_val = x_val.astype('float128') / 255.0

encoder = OneHotEncoder(sparse_output=False)

# Fit and transform the target values using OneHotEncoder
y_train = encoder.fit_transform(y_train.reshape(-1, 1))
print(y_train[0])
y_val = encoder.transform(y_val.reshape(-1, 1))
y_test = encoder.transform(y_test.reshape(-1, 1))

# Flatten the images
x_train = x_train.reshape((-1, 28 * 28))
print(x_train.shape)
x_val = x_val.reshape((-1, 28 * 28))
x_test = x_test.reshape((-1, 28 * 28))


(60000, 28, 28)
(48000, 28, 28)
[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
(48000, 784)


In [27]:
class Activations :
  def sigmoid(self,x) :
    if(x>200):
      return 1
    if(x<-200):
      return 0
    else :
      return 1/(1 + math.exp(-x))

  def tanh(self,x) :
      t1 = math.exp(x)
      t2  = math.exp(-x)
      return (t1-t2)/(t1 + t2)

  def g1(self,a):
    n = len(a)
    h = np.array(a)
    for i in range(n) :
      h[i] = self.sigmoid(a[i])
    return h

  def g2(self,a):
    n = len(a)
    h = np.array(a)
    for i in range(n) :
      h[i] = self.tanh(a[i])
    return h

  def g3(self,a):
    n = len(a)
    h = np.array(a)
    for i in range(n) :
      h[i] = max(0,a[i])
    return h

  def SoftMax(self,a):
    max_a = np.max(a)
    exp_a = np.exp(a - max_a)
    sum_exp_a = np.sum(exp_a)
    y = exp_a / sum_exp_a
    return y

  '''def SoftMax(self,a) :
    erx = np.array(a)
    sum = 0
    for i in range(len(a)):
      erx[i] = math.exp(a[i])
      sum = sum + erx[i]
    y = np.array(a)
    for i in range(len(a)):
      y[i] = erx[i]/sum
    return y'''


class Differential :
  def sig_dif(self,a):
    n = len(a)
    g_dash = np.empty(n)
    activ = Activations()
    for i in range(n) :
      g_x = activ.sigmoid(a[i])
      g_dash[i] = g_x*(1-g_x)
    return g_dash

  def tan_dif(self,a):
    n = len(a)
    g_dash = np.empty(n)
    activ = Activations()
    for i in range(n) :
      f_x = activ.tanh(a[i])
      g_dash[i] = 1 - (f_x ** 2)
    return g_dash

  def Rel_dif(self,a):
    n = len(a)
    h = np.empty(n)
    for i in range(n) :
      if a[i]>=0 :
        h[i] = 1
      else :
        h[i] = 0
    return h

In [28]:
class Initializer :

  def Initialize(self,hidden_layers,npl):
    self.W = [[]]    # list consisting of all the W's

    # Create and append each Wi matrix filled with zeros to the list
    self.W.append(np.zeros((npl,784),dtype=np.float128))         # input layer h0

    for _ in range(hidden_layers-1):
        self.W.append(np.zeros((npl,npl),dtype=np.float128))

    self.W.append(np.zeros((10,npl),dtype=np.float128))

    self.b = [[]]    # list consiting of all the b's

    # Create and append each bi vector filled with zeros to the list
    self.b.append(np.zeros(npl,dtype=np.float128))

    for _ in range(hidden_layers-1):
        self.b.append(np.zeros(npl,dtype=np.float128))

    self.b.append(np.zeros(10,dtype=np.float128))

    return self.W,self.b


  def Initialize2(self,hidden_layers,npl):
      self.W = [[]]    # list consisting of all the W's

      # Create and append each Wi matrix filled with random values to the list
      self.W.append(np.random.randn(npl, 784).astype(np.float128))         # input layer h0

      for _ in range(hidden_layers-1):
          self.W.append(np.random.randn(npl, npl).astype(np.float128))

      self.W.append(np.random.randn(10, npl).astype(np.float128))

      self.b = [[]]    # list consiting of all the b's

      # Create and append each bi vector filled with random values to the list
      self.b.append(np.random.randn(npl).astype(np.float128))

      for _ in range(hidden_layers-1):
          self.b.append(np.random.randn(npl).astype(np.float128))

      self.b.append(np.random.randn(10).astype(np.float128))
      return self.W,self.b

  def XavierIntializer(self,hidden_layers,npl):
    self.W = [[]]
    self.b = [[]]
    return W,b

In [29]:
class Arithmetic :
  def Add(self,u,v):
    for i in range(1,len(u)):
      u[i] = u[i] + v[i]
    return u

  def Subtract(self,v,dv,eta):
    for i in range(1,len(v)):
      v[i] = v[i] - (eta * dv[i])
    return v

  def RMSpropSubtract(self,v,dv,lv,eps,eta):
    for i in range(1,len(v)):
      ueta = eta/(np.sqrt(np.sum(lv[i])) + eps)
      v[i] = v[i] - (ueta * dv[i])
    return v

  def AdamSubtract(self,V,mV_hat,vV_hat,eps,eta):
    for i in range(1,len(V)):
      norm = np.linalg.norm(vV_hat[i])
      ueta = eta/(np.sqrt(norm) + eps)
      V[i] = V[i] - (ueta * mV_hat[i])
    return V

In [38]:
class Gradient_descent :
  def __init__(self, input_size, output_size, config):
    self.input_size = input_size
    self.output_size = output_size
    self.layers = config['layers']
    self.activation = config['activation']
    self.npl = config["neurons_per_layer"]
    self.eta = config["learning_rate"]
    self.batch = config["batch_size"]
    self.init = config["Initialization"]
    self.config = config

  def backward_propagation(self,A,H,W,b,y):
    L = self.layers

    delA = [[]]*(L+1)
    delW = [[]]*(L+1)
    delb = [[]]*(L+1)
    delh = [[]]*(L+1)

    delA[L] = -(y - H[L])

    for k in range(L,0,-1):
      delW[k] = np.outer(delA[k],H[k-1])
      delb[k] = delA[k]
      delh[k-1] = W[k].T @ delA[k]
      if k>1 :
        diff_vect = np.array(A[k-1])
        d = Differential()
        if self.activation == 'sigmoid':
          diff_vect = d.sig_dif(A[k-1])
        elif self.activation == 'tanh':
          diff_vect = d.tan_dif(A[k-1])
        else :
          diff_vect = d.Rel_dif(A[k-1])
        delA[k-1] = np.multiply(delh[k-1],diff_vect)

    return delW,delb

  def forward_propagation(self,W,b,layers,inpl):
    A = [[]]
    H = [inpl]
    activ = Activations()

    for i in range (1,layers) :
      a = b[i] + (W[i] @ H[i-1])

      A.append(a)
      h = np.array(a)

      if self.activation == 'sigmoid':   # Sigmoid activation function
        h = activ.g1(a)
      elif self.activation == 'tanh':   # tanh activation function
        h = activ.g2(a)
      else :            # ReLU activation function
        h = activ.g3(a)

      H.append(h)

    a = b[layers] + np.inner(W[layers],H[layers-1])
    A.append(a)
    y_hat = activ.SoftMax(a)
    H.append(y_hat)
    #print('y_hat = ',y_hat)
    return A,H

  def Stocastic_Gradient_descent(self) :
    epochs = self.config['epochs']
    W = []    # list consisting of all the W's
    b = []    # list consiting of all the b's

    I = Initializer()
    if(self.init == "random"):
      W,b = I.Initialize2(self.layers-1,self.npl)
    else:
      W,b = I.XavierIntializer(self.layers-1,self.npl)

    training_loss = []
    for i in range(epochs):
      delW,delb = I.Initialize(self.layers-1,self.npl)
      for j in range(len(x_train)): #len(trainX)

        h0 = x_train[j]
        A,H = self.forward_propagation(W,b,self.layers,h0)
        y = y_train[j]

        tdelW,tdelb = self.backward_propagation(A,H,W,b,y)
        PMA = Arithmetic()    # P - Plus, M - Minus , A - Arithmetic
        delW = PMA.Add(delW,tdelW)
        delb = PMA.Add(delb,tdelb)
        if((j+1) % self.batch == 0):
          W = PMA.Subtract(W,delW,self.eta)
          b = PMA.Subtract(b,delb,self.eta)
          delW,delb = I.Initialize(self.layers-1,self.npl)

    return W,b

  def Momentum_Gradient_descent(self) :
    epochs = self.config['epochs']
    W = []    # list consisting of all the W's
    b = []    # list consiting of all the b's

    I = Initializer()
    if(self.init == "random"):
      W,b = I.Initialize2(self.layers-1,self.npl)
    else:
      W,b = W,b = I.XavierIntializer(self.layers-1,self.npl)

    prev_uW,prev_ub = I.Initialize(self.layers-1,self.npl)
    beta = 0.9

    for i in range(epochs):
      delW,delb = I.Initialize(self.layers-1,self.npl)
      for j in range(len(x_train)):

        h0 = x_train[j]
        A,H = self.forward_propagation(W,b,self.layers,h0)
        y = y_train[j]

        temp_delW,temp_delb = self.backward_propagation(A,H,W,b,y)
        PMA = Arithmetic()
        delW = PMA.Add(delW,temp_delW)
        delb = PMA.Add(delb,temp_delb)

        if((j+1)%self.batch == 0):
          uW = [[]]
          ub = [[]]
          for k in range(1,len(prev_uW)):
            uW.append(beta*prev_uW[k] + delW[k])
            ub.append(beta*prev_ub[k] + delb[k])

          W = PMA.Subtract(W,uW,self.eta)
          b = PMA.Subtract(b,ub,self.eta)
          prev_ub = ub
          prev_uW = uW
          delW,delb = I.Initialize(self.layers-1,self.npl)

    return W,b

  def NAG_descent(self) :    # function(layers ,neurons per layer)
    epochs =  self.config['epochs']
    W = []    # list consisting of all the W's
    b = []    # list consiting of all the b's

    I = Initializer()
    if(self.init == "random"):
      W,b = I.Initialize2(self.layers-1,self.npl)
    else:
      W,b = W,b = I.XavierIntializer(self.layers-1,self.npl)
    prev_vW,prev_vb = I.Initialize(self.layers-1,self.npl)
    beta = 0.9

    for i in range(epochs):
      delW,delb = I.Initialize(self.layers-1,self.npl)
      vW = [[]]
      vb = [[]]
      for k in range(1,len(prev_vW)):
          vW.append(beta*prev_vW[k])
          vb.append(beta*prev_vb[k])

      for j in range(len(x_train)):
        h0 = x_train[j]
        #print(h0,len(h0))
        A,H = self.forward_propagation(W,b,self.layers,h0)
        y = y_train[j]

        ASA = Arithmetic()
        tempW = ASA.Subtract(W,vW,beta)
        tempb = ASA.Subtract(b,vb,beta)

        temp_delW,temp_delb = self.backward_propagation(A,H,tempW,tempb,y)
        delW = ASA.Add(delW,temp_delW)
        delb = ASA.Add(delb,temp_delb)

        if((j+1)%self.batch == 0):
          for k in range(len(prev_vW)):
            vW[k] += delW[k]
            vb[k] += delb[k]

          W = ASA.Subtract(W,vW,self.eta)
          b = ASA.Subtract(b,vb,self.eta)
          prev_vb = vb
          prev_vW = vW
          delW,delb = I.Initialize(self.layers-1,self.npl)

    return W,b

  def RMSprop(self) :
    epochs = self.config["epochs"]

    I = Initializer()
    if(self.init == "random"):
      W,b = I.Initialize2(self.layers-1,self.npl)
    else:
      W,b = W,b = I.XavierIntializer(self.layers-1,self.npl)

    vW,vb = I.Initialize(self.layers-1,self.npl)
    beta = 0.9
    eps = 1e-4
    batch_size = 50
    for i in range(epochs):
      delW,delb = I.Initialize(self.layers-1,self.npl)

      for j in range(len(x_train)): #len(trainX)
        h0 = x_train[j]
        A,H = self.forward_propagation(W,b,self.layers,h0)
        y = y_train[j]
        temp_delW,temp_delb = self.backward_propagation(A,H,W,b,y)
        PMA = Arithmetic()
        delW = PMA.Add(delW,temp_delW)
        delb = PMA.Add(delb,temp_delb)

        if((j+1)% self.batch == 0):
          for k in range(1,len(vW)):
            vW[k] = (beta * vW[k])+ ((1-beta)*(delW[k]**2))
            vb[k] = (beta * vb[k])+ ((1-beta)*(delb[k]**2))

          W = PMA.RMSpropSubtract(W,delW,vW,eps,self.eta)
          b = PMA.RMSpropSubtract(b,delb,vb,eps,self.eta)
          delW,delb = I.Initialize(self.layers-1,self.npl)

    return W,b

  def Adam(self) :
    epochs = self.config["epochs"]

    I = Initializer()
    if(self.init == "random"):
      W,b = I.Initialize2(self.layers-1,self.npl)
    else:
      W,b = W,b = I.XavierIntializer(self.layers-1,self.npl)

    vW,vb = I.Initialize(self.layers-1,self.npl)
    mW,mb = I.Initialize(self.layers-1,self.npl)
    beta1,beta2 = 0.9,0.999

    for i in range(epochs):
      delW,delb = I.Initialize(self.layers-1,self.npl)
      eps = 1e-10

      for j in range(len(x_train)): #len(trainX)
        h0 = x_train[j]
        A,H = self.forward_propagation(W,b,self.layers,h0)
        y = y_train[j]
        temp_delW,temp_delb = self.backward_propagation(A,H,W,b,y)
        PMA = Arithmetic()
        delW = PMA.Add(delW,temp_delW)
        delb = PMA.Add(delb,temp_delb)


        if((j+1)% self.batch == 0):
          vW_hat,vb_hat = I.Initialize(self.layers-1,self.npl)
          mW_hat,mb_hat = I.Initialize(self.layers-1,self.npl)

          for k in range(1,len(vW)):
            mW[k] = (beta1 * mW[k]) + ((1-beta1)*(delW[k]))

            mW_hat[k] = mW[k]/(1-np.power(beta1,i+1))

            mb[k] = (beta1 * mb[k]) + ((1-beta1)*(delb[k]))

            mb_hat[k] = mb[k]/(1-np.power(beta1,i+1))

            vW[k] = (beta2 * vW[k])+ ((1-beta2)*(delW[k]**2))

            vW_hat[k] = vW[k]/(1-np.power(beta2,i+1))

            vb[k] = (beta2 * vb[k])+ ((1-beta2)*(delb[k]**2))

            vb_hat[k] = vb[k]/(1-np.power(beta2,i+1))

          W = PMA.AdamSubtract(W,mW_hat,vW_hat,eps,self.eta)
          b = PMA.AdamSubtract(b,mb_hat,vb_hat,eps,self.eta)
          delW,delb = I.Initialize(self.layers-1,self.npl)

    return W,b

  def NAdam(self) :
    epochs = self.config["epochs"]

    I = Initializer()
    if(self.init == "random"):
      W,b = I.Initialize2(self.layers-1,self.npl)
    else:
      W,b = W,b = I.XavierIntializer(self.layers-1,self.npl)

    vW,vb = I.Initialize(self.layers-1,self.npl)
    mW,mb = I.Initialize(self.layers-1,self.npl)
    beta1,beta2 = 0.9,0.999

    for i in range(epochs):
      delW,delb = I.Initialize(self.layers-1,self.npl)
      eps = 1e-10

      for j in range(len(x_train)): #len(trainX)
        h0 = x_train[j]
        A,H = self.forward_propagation(W,b,self.layers,h0)
        y = y_train[j]
        temp_delW,temp_delb = self.backward_propagation(A,H,W,b,y)
        PMA = Arithmetic()
        delW = PMA.Add(delW,temp_delW)
        delb = PMA.Add(delb,temp_delb)


        if((j+1)% self.batch == 0):
          vW_hat,vb_hat = I.Initialize(self.layers-1,self.npl)
          mW_hat,mb_hat = I.Initialize(self.layers-1,self.npl)
          uW_hat,ub_hat = I.Initialize(self.layers-1,self.npl)

          for k in range(1,len(vW)):
            mW[k] = (beta1 * mW[k]) + ((1-beta1)*(delW[k]))

            mW_hat[k] = mW[k]/(1-np.power(beta1,i+1))

            mb[k] = (beta1 * mb[k]) + ((1-beta1)*(delb[k]))

            mb_hat[k] = mb[k]/(1-np.power(beta1,i+1))

            vW[k] = (beta2 * vW[k])+ ((1-beta2)*(delW[k]**2))

            vW_hat[k] = vW[k]/(1-np.power(beta2,i+1))

            vb[k] = (beta2 * vb[k])+ ((1-beta2)*(delb[k]**2))

            vb_hat[k] = vb[k]/(1-np.power(beta2,i+1))

            uW_hat[k] = (beta1*mW_hat[k]) + (((1-beta1)/(1-(beta1)**(i+1)))*delW[k])

            ub_hat[k] = (beta1*mb_hat[k]) + (((1-beta1)/(1-(beta1)**(i+1)))*delb[k])


          W = PMA.AdamSubtract(W,uW_hat,vW_hat,eps,self.eta)
          b = PMA.AdamSubtract(b,ub_hat,vb_hat,eps,self.eta)
          delW,delb = I.Initialize(self.layers-1,self.npl)

    return W,b

  def predict_class(self,W,b,layers,inpl,y):
    A = [[]]
    H = [inpl]
    activ = Activations()

    for i in range (1,layers) :
      innprod = np.inner(W[i],H[i-1])
      a = b[i] + innprod
      A.append(a)
      h = np.array(a)
      if self.activation == 'sigmoid':   # Sigmoid activation function
        h = activ.g1(a)
      elif self.activation == 'tanh':   # tanh activation function
        h = activ.g2(a)
      else :            # ReLU activation function
        h = activ.g3(a)
      H.append(h)

    a = b[layers] + np.inner(W[layers],H[layers-1])
    y_hat = activ.SoftMax(a)
    #print(y_hat,y)
    i1 = np.argmax(y)
    i2 = np.argmax(y_hat)
    #print(i1,i2)
    if i1 == i2 :
      return True
    else :
      return False
    #print(sum(y_hat))


  def predict(self,W,b):
    #print(W,b)
    count = 0
    for i in range(len(x_test)):
      h0 = x_test[i]
      y = y_test[i]
      if(self.predict_class(W,b,self.layers,h0,y)):
        count = count + 1

    print(count)

config = {
    "epochs" : 5,
    "layers": 3,
    "activation": "sigmoid",
    "neurons_per_layer" : 64,
    "learning_rate" : 0.1,
    "batch_size" : 1,
    "Initialization" : "random"
}


In [39]:
Model = Gradient_descent(784,10,config)

In [35]:
W,b = Model.Adam()
Model.predict(W,b)

8093


In [ ]:
W,b = Model.Stocastic_Gradient_descent();
Model.predict(W,b)

7949


In [40]:
W,b = Model.NAG_descent();
Model.predict(W,b)

1000


In [ ]:
def main():
    '''
    WandB calls main function each time with differnet combination.

    We can retrive the same and use the same values for our hypermeters.

    '''

    with wandb.init() as run:

        run_name="-ac_"+wandb.config.activation+"-hs"+str(wandb.config.hidden_size)
        wandb.run.name=run_name
        #obj=NN(wandb.config['num_layers'],wandb.config['hidden_size'])

        model = SimpleNN(784,10,wandb.config)
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=0.01,betas=(0.9, 0.999))
        model.train( model,criterion,optimizer,x_train,y_train,x_val,y_val)


wandb.agent(sweep_id, function=main,count=10) # calls main function for count number of times.
wandb.finish()